In [1]:
########################################################################################################################
# What it does: Sets up a local AI assistant that runs entirely on your own computer, 
#         with built-in research capabilities for ArXiv and Wikipedia searches.
# Key Benefits:
#        Complete Privacy: No data leaves your machine - perfect for sensitive competitive intelligence work
#        Zero API Costs: No per-query charges like ChatGPT/Claude
#        Research-Ready: Automatically finds and cites academic papers and references
#        Enterprise-Safe: Meets security requirements for handling proprietary information
# Core Value: This gives you the power of ChatGPT-level AI for your sales enablement and competitive analysis work, 
#        but with complete data control and no ongoing costs - exactly what Anaconda's enterprise customers need.
#######################################################################################################################

import os
import requests
import asyncio
import time
import logging
from dataclasses import dataclass
from datetime import datetime
from typing import List, Optional, Dict, Any
from IPython.display import display, Markdown, HTML

@dataclass
class LocalSettings:
    MODEL_API_URL: str = "http://127.0.0.1:8080"
    MODEL_NAME: str = "Meta-Llama-3-8B-Instruct_Q4_K_M.gguf"
    TEMPERATURE: float = 0.3
    MAX_TOKENS: int = 1500
    MAX_CONTEXT_LENGTH: int = 4096
    TOP_P: float = 0.9
    TOP_K: int = 40
    REPEAT_PENALTY: float = 1.1
    ENABLE_ARXIV: bool = True
    ENABLE_WIKIPEDIA: bool = True
    MAX_SEARCH_RESULTS: int = 10
    MAX_SOURCES_PER_QUERY: int = 7
    REQUESTS_PER_MINUTE: int = 20
    DEFAULT_CITATION_STYLE: str = "apa"
    DEFAULT_MAX_SOURCES: int = 5

settings = LocalSettings()

print("Configuration loaded:")
print(f"   Model API: {settings.MODEL_API_URL}")
print(f"   Model: {settings.MODEL_NAME}")
print(f"   Default Max Sources: {settings.DEFAULT_MAX_SOURCES}")

Configuration loaded:
   Model API: http://127.0.0.1:8080
   Model: Meta-Llama-3-8B-Instruct_Q4_K_M.gguf
   Default Max Sources: 5


In [2]:
############################################################################################################################
# What it does: Creates a LocalLLM class that connects to and manages your local Llama 3 AI model, 
#        with specialized methods for research tasks like generating summaries and evaluating source relevance.
# Key Benefits:
#      Plug-and-Play: Automatically detects and connects to your local AI model with health checks
#      Research-Optimized: Built-in functions for creating research reports and ranking source quality
#      Robust Fallbacks: Continues working even if the AI model goes offline (uses keyword matching)
#      Multiple API Support: Tries different endpoints to ensure compatibility with various local AI setups
# Core Value: This is the "brain" of your local AI system - it handles all the technical complexity of talking to your 
#      local model so you can focus on getting research insights for competitive intelligence and sales enablement without 
#      worrying about API failures or connection issues.
# Essentially, it makes your local AI as reliable and easy to use as a cloud service, but with complete data privacy.
############################################################################################################################


class LocalLLM:
    def __init__(self):
        self.api_url = settings.MODEL_API_URL
        self.model_name = settings.MODEL_NAME
        self.model_loaded = False
        self._test_connection()
        
    def _test_connection(self):
        try:
            response = requests.get(f"{self.api_url}/health", timeout=5)
            if response.status_code == 200:
                self.model_loaded = True
                print("✅ Connected to your local Llama 3 model API!")
                print(f"   Model: {self.model_name}")
                print(f"   Server: {self.api_url}")
            else:
                print(f"⚠️  API responded with status: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"❌ Could not connect to model API at {self.api_url}")
            print("   Make sure your Anaconda AI Navigator model is running")
    
    def generate_research_summary(self, query: str, sources_text: str, max_length: int = 1500) -> str:
        if not self.model_loaded:
            return "❌ Model API not available. Please start your model in Anaconda AI Navigator."
        
        system_message = "You are a professional research assistant. Create comprehensive, well-structured research reports."
        user_prompt = f"""Research Query: {query}

Available Sources:
{sources_text}

Please create a comprehensive research report that addresses the query directly.

Research Report:"""

        return self._generate_response(user_prompt, system_message, max_length)
    
    def evaluate_source_relevance(self, query: str, source_content: str) -> float:
        if not self.model_loaded:
            # Keyword matching fallback
            query_words = set(query.lower().split())
            content_words = set(source_content.lower().split())
            overlap = len(query_words.intersection(content_words))
            return min(overlap / len(query_words), 1.0) if query_words else 0.0
        
        system_message = "Rate source relevance to a query on a scale of 0.0 to 1.0. Respond with ONLY the numerical score."
        user_prompt = f"Query: {query}\nSource: {source_content[:1000]}...\nRelevance Score:"
        
        try:
            response = self._generate_response(user_prompt, system_message, 50)
            import re
            match = re.search(r'(\d+\.?\d*)', response)
            if match:
                score = float(match.group(1))
                return max(0.0, min(1.0, score))
        except:
            pass
        
        # Fallback to keyword matching
        query_words = set(query.lower().split())
        content_words = set(source_content.lower().split())
        overlap = len(query_words.intersection(content_words))
        return min(overlap / len(query_words), 1.0) if query_words else 0.0
    
    def _generate_response(self, prompt: str, system_message: str = None, max_tokens: int = None) -> str:
        if not self.model_loaded:
            return "❌ Model API not connected"
        
        max_tokens = max_tokens or settings.MAX_TOKENS
        
        try:
            if system_message:
                formatted_prompt = f"<|start_header_id|>system<|end_header_id|>\n{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
            else:
                formatted_prompt = f"<|start_header_id|>user<|end_header_id|>\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
            
            payload = {
                "prompt": formatted_prompt,
                "max_tokens": max_tokens,
                "temperature": settings.TEMPERATURE,
                "top_p": settings.TOP_P,
                "top_k": settings.TOP_K,
                "repeat_penalty": settings.REPEAT_PENALTY,
                "stop": ["<|eot_id|>", "<|end_of_text|>"]
            }
            
            endpoints = ["/v1/completions", "/completions", "/generate", "/api/generate"]
            
            for endpoint in endpoints:
                try:
                    response = requests.post(
                        f"{self.api_url}{endpoint}",
                        json=payload,
                        timeout=60,
                        headers={'Content-Type': 'application/json'}
                    )
                    
                    if response.status_code == 200:
                        result = response.json()
                        if 'choices' in result:
                            return result['choices'][0].get('text', '').strip()
                        elif 'response' in result:
                            return result['response'].strip()
                        elif 'generated_text' in result:
                            return result['generated_text'].strip()
                        elif 'text' in result:
                            return result['text'].strip()
                except:
                    continue
            
            return "❌ Could not get response from model API"
                
        except Exception as e:
            return f"❌ Error generating response: {str(e)}"

# Initialize LLM
llm = LocalLLM()


✅ Connected to your local Llama 3 model API!
   Model: Meta-Llama-3-8B-Instruct_Q4_K_M.gguf
   Server: http://127.0.0.1:8080


In [3]:
###############################################################################################################################
#  What it does: Defines data containers that organize and structure all your research information 
#          individual sources (papers, articles) and complete research results with citations.
# Key Benefits:
#         Clean Organization: Each source tracks title, URL, content, relevance score, and publication date in one place
#         Quality Scoring: Built-in relevance scoring to rank sources by importance to your query
#         Complete Research Packages: Bundles everything together - query, report, sources, citations, and confidence scores
#         Easy Tracking: Automatically records research time and maintains citation lists
# Core Value: This creates a professional research database structure that turns messy web searches into organized, 
#        citable research reports - perfect for creating those polished competitive analysis documents and sales battle cards 
#        that look enterprise-ready.
# Think of it as your research filing system that ensures nothing gets lost and everything is properly documented and scored.
################################################################################################################################

    
class Source:
    def __init__(self, title, url, content, relevance_score, source_type="web", publish_date=None):
        self.title = title
        self.url = url
        self.content = content
        self.relevance_score = relevance_score
        self.source_type = source_type
        self.publish_date = publish_date

@dataclass
class ResearchResult:
    query: str
    report: str
    sources: List[Source]
    citations: List[str]
    confidence_score: float
    research_time: float

print("Data structures defined")

Data structures defined


In [4]:
###############################################################################################################################
# What it does: Creates a multi-source search engine that simultaneously queries arXiv (academic papers) and Wikipedia, 
#      then combines results into structured, deduplicated source lists.
# Key Benefits:
#      Dual-Source Power: Gets both cutting-edge academic research AND accessible explanations in one search
#      Bulletproof Design: Automatically detects available search engines and gracefully handles failures
#      Fast Performance: Uses async searching to query multiple sources simultaneously
#      Clean Results: Removes duplicates and returns properly structured data ready for analysis
# Core Value: This gives you comprehensive research coverage for your competitive intelligence work,
#      academic papers for technical depth (perfect for understanding new AI/ML developments that competitors might be using) 
#      plus Wikipedia for context and background.
# Essential for creating those detailed technical battle cards where you need both the latest research insights and 
#     clear explanations that sales teams can actually use.
#################################################################################################################################
                                                                                                                                                                                                                             
                                                                                                                                                                                                                             
class SearchTool:
    def __init__(self):
        self.engines = {}
        self._check_available_engines()
    
    def _check_available_engines(self):
        try:
            import arxiv
            self.engines['arxiv'] = True
            print("✅ arXiv academic search available")
        except ImportError:
            self.engines['arxiv'] = False
            print("⚠️  arXiv not available")
        
        try:
            import wikipedia
            self.engines['wikipedia'] = True
            print("✅ Wikipedia search available")
        except ImportError:
            self.engines['wikipedia'] = False
            print("⚠️  Wikipedia not available")
    
    async def search(self, query: str, max_results: int = 10) -> List[Source]:
        print(f"🔍 Searching for: {query}")
        all_sources = []
        
        if self.engines.get('arxiv') and settings.ENABLE_ARXIV:
            academic_sources = await self._search_arxiv(query, max_results // 2)
            all_sources.extend(academic_sources)
        
        if self.engines.get('wikipedia') and settings.ENABLE_WIKIPEDIA:
            wiki_sources = await self._search_wikipedia(query, max_results // 2)
            all_sources.extend(wiki_sources)
        
        if not all_sources:
            print("⚠️  No search engines available - using demo data")
            return self._get_demo_sources(query)
        
        return self._remove_duplicates(all_sources)[:max_results]
    
    async def _search_arxiv(self, query: str, max_results: int) -> List[Source]:
        sources = []
        try:
            import arxiv
            client = arxiv.Client()
            search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)
            
            for paper in client.results(search):
                source = Source(
                    title=paper.title,
                    url=paper.entry_id,
                    content=paper.summary,
                    relevance_score=0.9,
                    source_type="academic",
                    publish_date=paper.published.strftime("%Y-%m-%d") if paper.published else None
                )
                sources.append(source)
            
            print(f"   ✅ Found {len(sources)} academic papers")
        except Exception as e:
            print(f"   ❌ Academic search failed: {e}")
        
        return sources
    
    async def _search_wikipedia(self, query: str, max_results: int) -> List[Source]:
        sources = []
        try:
            import wikipedia
            search_results = wikipedia.search(query, results=max_results)
            
            for title in search_results:
                try:
                    page = wikipedia.page(title)
                    source = Source(
                        title=page.title,
                        url=page.url,
                        content=page.summary,
                        relevance_score=0.7,
                        source_type="encyclopedia"
                    )
                    sources.append(source)
                except:
                    continue
            
            print(f"   ✅ Found {len(sources)} Wikipedia articles")
        except Exception as e:
            print(f"   ❌ Wikipedia search failed: {e}")
        
        return sources
    
    def _get_demo_sources(self, query: str) -> List[Source]:
        return [
            Source(
                title=f"Introduction to {query}",
                url="https://demo.example.com/intro",
                content=f"This is an introductory overview of {query}. This demo content shows what real research sources would look like.",
                relevance_score=0.7,
                source_type="web"
            )
        ]
    
    def _remove_duplicates(self, sources: List[Source]) -> List[Source]:
        seen_urls = set()
        unique_sources = []
        
        for source in sources:
            if source.url not in seen_urls:
                seen_urls.add(source.url)
                unique_sources.append(source)
        
        return unique_sources

search_tool = SearchTool()

✅ arXiv academic search available
✅ Wikipedia search available


In [5]:
###################################################################################################################################################
# What it does: The main research orchestrator that runs a complete research pipeline, searches multiple sources, evaluates quality, 
#      generates professional reports with citations, and provides confidence scoring.
# Key Benefits:
#     Fully Automated: Takes a simple query and delivers a complete research report with zero manual work
#     Quality Control: Automatically ranks sources by relevance and selects only the best ones
#     Professional Output: Generates properly cited reports with confidence scores for credibility
#     Bulletproof Reliability: Multiple fallback systems ensure you always get results even if components fail
#     Research Memory: Tracks all past research for easy reference and follow-up analysis
# Core Value: This is your one-click competitive intelligence generator - just ask a question like "What are Anaconda's latest security features?" 
#     and get back a professional research report with academic sources, Wikipedia context, proper citations, and a confidence score.
# Perfect for rapidly creating those detailed competitive battle cards and technical analysis documents that would normally 
#     take hours of manual research.
#####################################################################################################################################################


class ResearchAgent:
    def __init__(self):
        self.llm = llm
        self.search_tool = search_tool
        self.research_history = []
        print("Research Agent initialized and ready!")
    
    async def research(self, query: str, max_sources: int = None) -> ResearchResult:
        start_time = time.time()
        max_sources = max_sources or settings.DEFAULT_MAX_SOURCES
        
        print(f"\nStarting research on: {query}")
        print("=" * 50)
        
        try:
            # Step 1: Search for sources
            print("Step 1: Searching for sources...")
            sources = await self.search_tool.search(query, max_sources)
            
            if not sources:
                return ResearchResult(
                    query=query,
                    report="❌ No sources found for this query.",
                    sources=[],
                    citations=[],
                    confidence_score=0.0,
                    research_time=time.time() - start_time
                )
            
            print(f"   Found {len(sources)} sources")
            
            # Step 2: Evaluate relevance
            print("Step 2: Evaluating source relevance...")
            for i, source in enumerate(sources):
                print(f"   Evaluating source {i+1}/{len(sources)}...")
                if self.llm.model_loaded:
                    try:
                        relevance = self.llm.evaluate_source_relevance(query, source.content)
                        source.relevance_score = relevance
                    except:
                        pass
            
            # Step 3: Filter best sources
            best_sources = sorted(sources, key=lambda s: s.relevance_score, reverse=True)[:max_sources]
            print(f"   Selected {len(best_sources)} high-quality sources")
            
            # Step 4: Generate report
            print("Step 3: Generating research report...")
            report = await self._generate_report(query, best_sources)
            
            # Step 5: Generate citations
            print("Step 4: Creating citations...")
            citations = self._generate_citations(best_sources)
            
            # Step 6: Calculate confidence
            confidence = self._calculate_confidence(best_sources)
            research_time = time.time() - start_time
            
            result = ResearchResult(
                query=query,
                report=report,
                sources=best_sources,
                citations=citations,
                confidence_score=confidence,
                research_time=research_time
            )
            
            self.research_history.append(result)
            
            print(f"✅ Research completed in {research_time:.1f} seconds")
            print(f"   Confidence Score: {confidence:.2f}")
            print("=" * 50)
            
            return result
            
        except Exception as e:
            print(f"❌ Research failed: {e}")
            return ResearchResult(
                query=query,
                report=f"❌ Research failed due to error: {str(e)}",
                sources=[],
                citations=[],
                confidence_score=0.0,
                research_time=time.time() - start_time
            )
    
    async def _generate_report(self, query: str, sources: List[Source]) -> str:
        if not sources:
            return "No sources available to generate report."
        
        sources_text = ""
        for i, source in enumerate(sources, 1):
            sources_text += f"\nSource {i} - {source.title} ({source.source_type}):\n"
            sources_text += f"{source.content[:1000]}...\n"
            sources_text += f"URL: {source.url}\n"
            sources_text += "-" * 40 + "\n"
        
        if self.llm.model_loaded:
            try:
                report = self.llm.generate_research_summary(query, sources_text)
                return report
            except Exception as e:
                print(f"   ⚠️  LLM report generation failed: {e}")
        
        # Fallback: Simple summary
        fallback_report = f"# Research Report: {query}\n\n"
        fallback_report += f"Based on {len(sources)} sources, here are the key findings:\n\n"
        
        for i, source in enumerate(sources, 1):
            fallback_report += f"## Finding {i}: {source.title}\n"
            fallback_report += f"**Source:** {source.source_type.title()}\n"
            fallback_report += f"**Relevance:** {source.relevance_score:.2f}\n\n"
            fallback_report += f"{source.content[:500]}...\n\n"
        
        return fallback_report
    
    def _generate_citations(self, sources: List[Source]) -> List[str]:
        citations = []
        for source in sources:
            if source.source_type == "academic":
                citation = f"{source.title}. arXiv. Retrieved from {source.url}"
            elif source.source_type == "encyclopedia":
                citation = f"{source.title}. Wikipedia. Retrieved from {source.url}"
            else:
                citation = f"{source.title}. Retrieved from {source.url}"
            citations.append(citation)
        return citations
    
    def _calculate_confidence(self, sources: List[Source]) -> float:
        if not sources:
            return 0.0
        
        avg_relevance = sum(s.relevance_score for s in sources) / len(sources)
        source_types = len(set(s.source_type for s in sources))
        diversity_score = min(source_types / 3.0, 1.0)
        count_score = min(len(sources) / 5.0, 1.0)
        academic_count = sum(1 for s in sources if s.source_type == "academic")
        academic_bonus = min(academic_count / len(sources), 0.2)
        
        confidence = (avg_relevance * 0.4 + diversity_score * 0.2 + count_score * 0.2 + academic_bonus * 0.2)
        return min(confidence, 1.0)
    
    def get_research_history(self) -> List[ResearchResult]:
        return self.research_history.copy()

agent = ResearchAgent()

Research Agent initialized and ready!


In [6]:
###############################################################################################################################
# What it does: Creates professional display functions that format your research results with styled HTML output, 
#       organized sections, and clickable links - perfect for Jupyter notebooks.
# Key Benefits:
#       Executive-Ready Presentation: Transforms raw data into polished, professional reports with proper formatting
#       Easy Scanning: Color-coded sections, clear headings, and an organized layout for quick information absorption
#       Clickable Sources: Direct links to all source materials for easy verification and follow-up
#       System Monitoring: Real-time status dashboard showing what's working and what's not
# Core Value: This turns your research output into screenshot-ready competitive intelligence reports that you can immediately 
#       drop into presentations, battle cards, or share with sales teams.
# No more messy text dumps - everything looks like it came from a professional research firm, complete with confidence scores 
#       and proper source attribution that builds credibility with stakeholders.
#################################################################################################################################


def display_research_result(result: ResearchResult):
    # Research Summary
    summary_html = f"""
    <div style="background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0; border-left: 4px solid #007bff;">
        <h3 style="color: #007bff; margin-bottom: 15px;">Research Summary</h3>
        <p><strong>Query:</strong> {result.query}</p>
        <p><strong>Research Time:</strong> {result.research_time:.1f} seconds</p>
        <p><strong>Confidence Score:</strong> {result.confidence_score:.2f}/1.0</p>
        <p><strong>Sources Found:</strong> {len(result.sources)}</p>
        <p><strong>Completed:</strong> {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
    </div>
    """
    display(HTML(summary_html))
    
    # Research Report
    display(Markdown("## Research Report"))
    display(Markdown(result.report))
    
    # Sources
    if result.sources:
        display(Markdown(f"## Sources ({len(result.sources)})"))
        for i, source in enumerate(result.sources, 1):
            source_html = f"""
            <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #28a745;">
                <h4 style="color: #333; margin-bottom: 5px;">{i}. {source.title}</h4>
                <p style="color: #666; font-size: 14px; margin: 5px 0;">
                    <strong>Type:</strong> {source.source_type.title()} | 
                    <strong>Relevance:</strong> {source.relevance_score:.2f} | 
                    <a href="{source.url}" target="_blank">View Source</a>
                </p>
            </div>
            """
            display(HTML(source_html))
    
    # Citations
    if result.citations:
        display(Markdown("## Citations"))
        for i, citation in enumerate(result.citations, 1):
            display(Markdown(f"{i}. {citation}"))

def show_system_status():
    status_html = f"""
    <div style="background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">
        <h3 style="color: #333; margin-bottom: 15px;">System Status</h3>
        <p><strong>LLM Connected:</strong> {'✅ Yes' if llm.model_loaded else '❌ No'}</p>
        <p><strong>arXiv Search:</strong> {'✅ Available' if search_tool.engines.get('arxiv') else '❌ Not Available'}</p>
        <p><strong>Wikipedia Search:</strong> {'✅ Available' if search_tool.engines.get('wikipedia') else '❌ Not Available'}</p>
        <p><strong>Model API:</strong> {settings.MODEL_API_URL}</p>
        <p><strong> Model:</strong> {settings.MODEL_NAME}</p>
    </div>
    """
    display(HTML(status_html))

print("Display functions ready!")

Display functions ready!


In [7]:
#########################################################################################################################
# What it does: Executes the complete research pipeline - runs your query through the entire system and displays 
#      the final formatted research report with sources, citations, and confidence scores.
# Key Benefits:
#      One-Click Research: Single command generates a complete competitive intelligence report
#      Customizable Scope: Easily adjust the number of sources to control depth vs. speed
#      Immediate Results: Async execution provides fast turnaround for time-sensitive analysis
#      Ready-to-Share Output: Professional formatting perfect for stakeholder presentations
# Core Value: This is your research button - the moment where all the technical complexity disappears and you get 
#      exactly what you need for competitive analysis: a professional research report on any topic 
#      (like "Anaconda's security features" or "enterprise Python governance trends") in seconds instead of hours.
# Perfect for those urgent competitive intelligence requests where you need credible, cited analysis fast
##########################################################################################################################


RESEARCH_QUERY = "machine learning transformers"
MAX_SOURCES = 5

print(f"Researching: {RESEARCH_QUERY}")
result = await agent.research(RESEARCH_QUERY, max_sources=MAX_SOURCES)
display_research_result(result)


Researching: machine learning transformers

Starting research on: machine learning transformers
Step 1: Searching for sources...
🔍 Searching for: machine learning transformers
   ❌ Academic search failed: Page request resulted in HTTP 301: None (http://export.arxiv.org/api/query?search_query=machine+learning+transformers&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=2)
   ✅ Found 2 Wikipedia articles
   Found 2 sources
Step 2: Evaluating source relevance...
   Evaluating source 1/2...
   Evaluating source 2/2...
   Selected 2 high-quality sources
Step 3: Generating research report...
Step 4: Creating citations...
✅ Research completed in 47.1 seconds
   Confidence Score: 0.50


## Research Report

**Title:** An Exploration of Machine Learning Transformers and Attention Mechanisms

**Abstract:**
This report provides an in-depth analysis of machine learning transformers, a neural network architecture that has gained significant attention in recent years. We will delve into the concept of transformers, their advantages over traditional recurrent neural networks (RNNs), and the role of attention mechanisms in this architecture.

**Introduction:**
Machine learning transformers have revolutionized the field of natural language processing by enabling parallel processing and reducing training time. The transformer architecture was first proposed in the 2017 paper "Attention Is All You Need" by researchers at Google. This report aims to provide a comprehensive overview of machine learning transformers, their components, and the attention mechanism that underlies their functionality.

**Transformer Architecture:**
A transformer is a neural network architecture based on the multi-head attention mechanism (Source 1). The architecture consists of an encoder and a decoder. The encoder converts text into numerical representations called tokens, which are then converted into vectors via lookup from a word embedding table. Each token is contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism.

The transformer architecture has several advantages over traditional RNNs:

1. **Parallel Processing:** Transformers can process input sequences in parallel, reducing training time and making them more efficient than RNNs.
2. **No Recurrent Units:** Unlike RNNs, transformers do not require recurrent units, which reduces the complexity of the model and makes it easier to train.

**Attention Mechanism:**
The attention mechanism is a key component of the transformer architecture (Source 2). It determines the importance of each component in a sequence relative to the other components. In natural language processing, this means assigning "soft" weights to each word in a sentence based on its relevance to the context.

The attention mechanism has several benefits:

1. **Contextualization:** Attention allows tokens to be contextualized within the scope of the context window, enabling the model to capture long-range dependencies.
2. **Weighted Summation:** The attention mechanism enables the model to compute a weighted sum of token embeddings, which improves the accuracy of the model.

**Conclusion:**
Machine learning transformers have revolutionized the field of natural language processing by providing a parallel and efficient architecture for processing sequential data. The attention mechanism is a key component of this architecture, enabling the model to capture long-range dependencies and contextualize tokens within the scope of the context window. This report provides a comprehensive overview of machine learning transformers and their components, highlighting their advantages over traditional RNNs.

**Recommendations:**
Future research should focus on exploring new applications of machine learning transformers, such as image processing and speech recognition. Additionally, researchers should investigate ways to improve the attention mechanism, such as incorporating external knowledge or using more advanced attention techniques.

**References:**

1. "Transformer (deep learning architecture)" (Source 1)
2. "Attention (machine learning)" (Source 2)

**Appendices:**
* Additional information on transformer architectures and their applications
* Code snippets for implementing transformers in popular machine learning frameworks

## Sources (2)

## Citations

1. Transformer (deep learning architecture). Wikipedia. Retrieved from https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)

2. Attention (machine learning). Wikipedia. Retrieved from https://en.wikipedia.org/wiki/Attention_(machine_learning)

In [8]:
#####################################################################################################################################
# What it does: Displays a real-time health check dashboard showing the connection status of your local AI model, 
#     search engines, and system configuration.
# Key Benefits:
#     Instant Diagnostics: Quickly see what's working and what needs attention
#     Troubleshooting Made Easy: Color-coded status indicators pinpoint issues immediately
#     System Overview: Shows your current model and API settings at a glance
#     Zero Guesswork: Eliminates "is it working?" uncertainty before running research
# Core Value: This is your system dashboard. It is essential to ensure everything is running smoothly before diving into competitive 
#     intelligence research. Saves you from wasting time on queries when components are down and gives you confidence that your 
#     research results are coming from all available sources.
# Perfect for quick status checks during demos or when preparing for important research sessions.
#######################################################################################################################################


show_system_status()

In [9]:
#########################################################################################################################
# What it does: Displays a searchable log of all your previous research queries with key metrics like confidence scores, 
#     source counts, and processing times in an organized timeline format.
# Key Benefits:
#     Research Trail: Never lose track of what you've already investigated
#     Performance Tracking: See which queries worked best based on confidence scores and source quality
#     Time Management: Track how long different types of research take to optimize your workflow
#     Easy Reference: Quickly revisit past competitive intelligence without re-running searches
# Core Value: This creates your competitive intelligence knowledge base, which enables a permanent record 
#     of all your research that helps you avoid duplicate work, track investigation patterns, 
#     and build a comprehensive database of competitive insights over time.
# Essential for maintaining institutional knowledge and ensuring you can always reference past analysis when 
#     building updated battle cards or presenting historical competitive trends to leadership.
#########################################################################################################################


history = agent.get_research_history()

if not history:
    print("No research history yet. Run a research query first!")
else:
    display(Markdown(f"## Research History ({len(history)} queries)"))
    
    for i, result in enumerate(history, 1):
        history_html = f"""
        <div style="background: #e8f4f8; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #17a2b8;">
            <h4 style="color: #333; margin-bottom: 10px;">{i}. {result.query}</h4>
            <p style="color: #666; font-size: 14px; margin: 5px 0;">
                <strong>Confidence:</strong> {result.confidence_score:.2f} | 
                <strong>Sources:</strong> {len(result.sources)} | 
                <strong>Time:</strong> {result.research_time:.1f}s
            </p>
        </div>
        """
        display(HTML(history_html))

## Research History (1 queries)

In [10]:
# What it does: Navigates to your research agent project folder and displays all the files and 
#     directories to confirm you're in the correct location for running the AI research system.
# Key Benefits:
#     File Path Verification: Ensures you're in the correct directory before running any code
#     Environment Setup: Prevents "file not found" errors when accessing project files
#     Project Overview: Shows all available files and components at a glance
#     Debugging Aid: Helps troubleshoot path-related issues quickly
# Core Value: This is your project initialization checkpoint - essential for ensuring your local AI research system 
#     can locate all its components (models, configuration files, data) before you begin any competitive intelligence work.
# Think of it as checking your toolkit before starting a job - ensures everything is where it should be, so your research queries run smoothly.


import os

# Change to your research_agent directory
os.chdir('/Users/smonroe/research_agent')

# Verify you're in the right place
print("Current directory:", os.getcwd())
print("\nDirectory contents:")
for item in os.listdir('.'):
    print(f"  {item}")

Current directory: /Users/smonroe/research_agent

Directory contents:
  environment.yml
  config
  tests
  models
  logs
  environment-ai-agent.yml
  .git
  main.py
  data
  outputs
  sam_web_interface.py
  research_agent
  conda-spec.txt
  src


In [11]:
# What it does: Creates a professional web application with a polished browser interface for your local AI research agent, 
#    complete with tabs for research, status monitoring, and history tracking.
# Key Benefits:
#     Demo-Ready Interface: Clean, professional design perfect for customer presentations and sales demos
#     Zero Setup Required: Automatically launches in your browser from Jupyter with one command
#     Multi-Tab Organization: Separate sections for research, system status, history, and documentation
#     Sample Query Library: Pre-loaded examples to help users get started quickly
#     Real-Time Feedback: Loading indicators, progress updates, and professional result formatting
# Core Value: This transforms your technical research system into a client-facing demo tool that showcases 
#     Anaconda's AI Platform capabilities. Perfect for sales presentations where you need to demonstrate local AI, secure governance, 
#     and enterprise-ready research capabilities without revealing any underlying complexity.
# Essentially, it's your competitive intelligence showcase that is professional enough to present to C-level executives 
#    while highlighting Anaconda's key differentiators around local deployment, data privacy, and enterprise security.
###################################################################################################################################



from flask import Flask, render_template_string, request, jsonify
import asyncio
import sys
import os
from datetime import datetime
import threading
import webbrowser
from time import sleep

# Fix for Jupyter
sys.path.append(os.getcwd())

# Import your research components
from src.agent import get_research_agent
from config.settings import settings

# Create Flask app
app = Flask(__name__)
research_agent = None

def initialize_agent():
    global research_agent
    if research_agent is None:
        try:
            research_agent = get_research_agent()
            return True, "Research agent initialized successfully!"
        except Exception as e:
            return False, f"Failed to initialize agent: {str(e)}"
    return True, "Research agent already initialized"

# Your HTML template (same as before)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title> Local Research Agent - Powered by Anaconda AI Platform </title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 20px 40px rgba(0,0,0,0.1);
            overflow: hidden;
        }
        .header {
            background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }
        .header h1 { font-size: 2.5em; margin-bottom: 10px; }
        .header p { font-size: 1.1em; opacity: 0.9; }
        .main-content { padding: 30px; }
        .tabs {
            display: flex;
            margin-bottom: 30px;
            border-bottom: 2px solid #f0f0f0;
        }
        .tab {
            padding: 15px 25px;
            cursor: pointer;
            border-bottom: 3px solid transparent;
            font-weight: 500;
            transition: all 0.3s;
        }
        .tab:hover { background: #f8f9fa; }
        .tab.active { border-bottom-color: #667eea; color: #667eea; }
        .tab-content { display: none; }
        .tab-content.active { display: block; }
        .research-form {
            background: #f8f9fa;
            padding: 25px;
            border-radius: 10px;
            margin-bottom: 30px;
        }
        .form-group { margin-bottom: 20px; }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: 600;
            color: #333;
        }
        input[type="text"], textarea, select {
            width: 100%;
            padding: 12px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            font-size: 16px;
            transition: border-color 0.3s;
        }
        input[type="text"]:focus, textarea:focus, select:focus {
            outline: none;
            border-color: #667eea;
        }
        .search-btn {
            background: linear-gradient(45deg, #667eea, #764ba2);
            color: white;
            border: none;
            padding: 15px 30px;
            font-size: 16px;
            border-radius: 8px;
            cursor: pointer;
            transition: transform 0.2s;
        }
        .search-btn:hover { transform: translateY(-2px); }
        .search-btn:disabled { opacity: 0.6; transform: none; cursor: not-allowed; }
        .results { margin-top: 30px; }
        .result-section {
            background: white;
            border: 1px solid #e0e0e0;
            border-radius: 10px;
            padding: 25px;
            margin-bottom: 20px;
            line-height: 1.6;
        }
        .result-section h3 {
            color: #333;
            margin-bottom: 20px;
            padding-bottom: 10px;
            border-bottom: 2px solid #f0f0f0;
            font-size: 1.3em;
        }
        .result-section h4 {
            color: #444;
            margin-top: 25px;
            margin-bottom: 15px;
            font-size: 1.1em;
            font-weight: 600;
        }
        .result-section ul {
            margin: 15px 0;
            padding-left: 40px;
        }
        .result-section ol {
            margin: 15px 0;
            padding-left: 40px;
        }
        .result-section li {
            margin-bottom: 12px;
            padding-left: 8px;
            line-height: 1.7;
        }
        .result-section ul li {
            list-style-type: disc;
            list-style-position: outside;
        }
        .result-section ol li {
            list-style-type: decimal;
            list-style-position: outside;
        }
        .result-section ul ul, .result-section ol ul {
            margin: 8px 0;
            padding-left: 25px;
        }
        .result-section ul ol, .result-section ol ol {
            margin: 8px 0;
            padding-left: 25px;
        }
        .result-section p {
            margin-bottom: 15px;
            line-height: 1.6;
        }
        .loading {
            text-align: center;
            padding: 40px;
            color: #666;
        }
        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 2s linear infinite;
            margin: 0 auto 20px;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .source-item {
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 10px;
            border-left: 4px solid #667eea;
        }
        .source-title {
            font-weight: 600;
            color: #333;
            margin-bottom: 5px;
        }
        .source-meta {
            color: #666;
            font-size: 14px;
        }
        .samples {
            background: #e8f4fd;
            padding: 20px;
            border-radius: 10px;
            margin-top: 20px;
        }
        .samples h4 {
            color: #1976d2;
            margin-bottom: 15px;
        }
        .sample-query {
            background: white;
            padding: 10px 15px;
            border-radius: 6px;
            margin-bottom: 8px;
            cursor: pointer;
            transition: background 0.2s;
        }
        .sample-query:hover {
            background: #f0f8ff;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>Local Research Agent Powered by Anaconda AI Platform </h1>
            <p>Powered by Meta-Llama-3-8B-Instruct via Anaconda AI Catalog</p>
            <p>Sources: arXiv Academic Papers + Wikipedia | Everything runs locally!</p>
        </div>
        
        <div class="main-content">
            <div class="tabs">
                <div class="tab active" onclick="showTab('research')">🔍 Research</div>
                <div class="tab" onclick="showTab('status')">📊 Status</div>
                <div class="tab" onclick="showTab('history')">📚 History</div>
                <div class="tab" onclick="showTab('about')">ℹ️ About</div>
            </div>
            
            <div id="research" class="tab-content active">
                <div class="research-form">
                    <div class="form-group">
                        <label for="query">Research Query</label>
                        <textarea id="query" rows="3" placeholder="Enter your research question (e.g., 'machine learning transformers', 'renewable energy benefits')"></textarea>
                    </div>
                    
                    <div class="form-group">
                        <label for="maxSources">Maximum Sources</label>
                        <select id="maxSources">
                            <option value="3">3 sources</option>
                            <option value="5" selected>5 sources</option>
                            <option value="7">7 sources</option>
                            <option value="10">10 sources</option>
                        </select>
                    </div>
                    
                    <button class="search-btn" onclick="startResearch()">Start Research</button>
                </div>
                
                <div class="samples">
                    <h4>Sample Queries</h4>
                    <div class="sample-query" onclick="setQuery('machine learning transformers')">Academic: "machine learning transformers"</div>
                    <div class="sample-query" onclick="setQuery('quantum computing algorithms')">Science: "quantum computing algorithms"</div>
                    <div class="sample-query" onclick="setQuery('renewable energy benefits')">General: "renewable energy benefits"</div>
                    <div class="sample-query" onclick="setQuery('artificial intelligence ethics')">Technology: "artificial intelligence ethics"</div>
                </div>
                
                <div id="results" class="results"></div>
            </div>
            
            <div id="status" class="tab-content">
                <div class="result-section">
                    <h3>📊 System Status</h3>
                    <div id="statusContent">Click "Refresh Status" to check system status...</div>
                    <br>
                    <button class="search-btn" onclick="checkStatus()">🔄 Refresh Status</button>
                </div>
            </div>
            
            <div id="history" class="tab-content">
                <div class="result-section">
                    <h3>📚 Research History</h3>
                    <div id="historyContent">Click "Refresh History" to view past research...</div>
                    <br>
                    <button class="search-btn" onclick="loadHistory()">🔄 Refresh History</button>
                </div>
            </div>
            
            <div id="about" class="tab-content">
                <div class="result-section">
                    <h3>About Local Research Agent</h3>
                    <p>This research agent combines packages from Anaconda's Secure Governance with policy enforcement with local secure models to provide comprehensive research capabilities.</p>
                    
                    <h4>Anaconda AI Platform Technology Stack</h4>
                    <ul>
                        <li><strong>Language Model:</strong> Via Anaconda's secure and governed model repository Meta-Llama-3-8B-Instruct (running locally)</li>
                        <li><strong>Package and Environment Management:</strong> Enabled by packages with Anaconda Secure Governance</li>
                        <li><strong>Academic Search:</strong> arXiv API for scientific papers</li>
                        <li><strong>General Knowledge:</strong> Wikipedia API for encyclopedic information</li>
                        <li><strong>Interface:</strong> Flask web framework for user-friendly interface</li>
                    </ul>
                    
                    <h4>How It Works</h4>
                    <ol>
                        <li><strong>Search Phase:</strong> Queries arXiv and Wikipedia for relevant sources</li>
                        <li><strong>Analysis Phase:</strong> Your local Llama 3 model evaluates source relevance</li>
                        <li><strong>Synthesis Phase:</strong> Generates comprehensive research reports</li>
                        <li><strong>Citation Phase:</strong> Creates properly formatted citations</li>
                    </ol>
                    
                    <h4>Privacy & Security</h4>
                    <ul>
                        <li>Everything runs locally on your machine</li>
                        <li>No data sent to external APIs (except for source searching)</li>
                        <li>Your queries and results stay private</li>
                        <li>No API keys required</li>
                    </ul>
                </div>
            </div>
        </div>
    </div>

    <script>
        function showTab(tabName) {
            const tabs = document.querySelectorAll('.tab-content');
            tabs.forEach(tab => tab.classList.remove('active'));
            const tabButtons = document.querySelectorAll('.tab');
            tabButtons.forEach(button => button.classList.remove('active'));
            document.getElementById(tabName).classList.add('active');
            event.target.classList.add('active');
        }
        
        function setQuery(query) {
            document.getElementById('query').value = query;
        }
        
        async function startResearch() {
            const query = document.getElementById('query').value.trim();
            const maxSources = document.getElementById('maxSources').value;
            const resultsDiv = document.getElementById('results');
            const searchBtn = document.querySelector('.search-btn');
            
            if (!query) {
                alert('Please enter a research query');
                return;
            }
            
            searchBtn.disabled = true;
            searchBtn.textContent = '🔄 Researching...';
            resultsDiv.innerHTML = '<div class="loading"><div class="spinner"></div><p>Conducting research on: "' + query + '"</p><p>This may take 30-60 seconds...</p></div>';
            
            try {
                const response = await fetch('/research', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ query: query, max_sources: parseInt(maxSources) })
                });
                
                const data = await response.json();
                
                if (data.success) {
                    resultsDiv.innerHTML = '<div class="result-section"><h3>Research Report</h3><div style="white-space: pre-wrap;">' + data.report + '</div></div><div class="result-section"><h3>📚 Sources (' + data.sources.length + ')</h3>' + data.sources.map((source, i) => '<div class="source-item"><div class="source-title">' + (i + 1) + '. ' + source.title + '</div><div class="source-meta">Type: ' + source.source_type + ' | Relevance: ' + source.relevance_score.toFixed(2) + ' | <a href="' + source.url + '" target="_blank">View Source</a></div></div>').join('') + '</div><div class="result-section"><h3>📖 Citations</h3>' + data.citations.map((citation, i) => '<p>' + (i + 1) + '. ' + citation + '</p>').join('') + '</div><div class="result-section"><h3>📊 Research Summary</h3><p><strong>Query:</strong> ' + data.query + '</p><p><strong>Research Time:</strong> ' + data.research_time.toFixed(1) + ' seconds</p><p><strong>Confidence Score:</strong> ' + data.confidence_score.toFixed(2) + '/1.0</p><p><strong>Sources Found:</strong> ' + data.sources.length + '</p></div>';
                } else {
                    resultsDiv.innerHTML = '<div class="result-section"><h3>❌ Error</h3><p>' + data.error + '</p></div>';
                }
            } catch (error) {
                resultsDiv.innerHTML = '<div class="result-section"><h3>❌ Error</h3><p>Failed to conduct research: ' + error.message + '</p></div>';
            } finally {
                searchBtn.disabled = false;
                searchBtn.textContent = 'Start Research';
            }
        }
        
        async function checkStatus() {
            const statusDiv = document.getElementById('statusContent');
            statusDiv.innerHTML = 'Loading status...';
            
            try {
                const response = await fetch('/status');
                const data = await response.json();
                statusDiv.innerHTML = '<pre>' + data.status + '</pre>';
            } catch (error) {
                statusDiv.innerHTML = 'Error loading status: ' + error.message;
            }
        }
        
        async function loadHistory() {
            const historyDiv = document.getElementById('historyContent');
            historyDiv.innerHTML = 'Loading history...';
            
            try {
                const response = await fetch('/history');
                const data = await response.json();
                historyDiv.innerHTML = '<pre>' + data.history + '</pre>';
            } catch (error) {
                historyDiv.innerHTML = 'Error loading history: ' + error.message;
            }
        }
        
        document.getElementById('query').addEventListener('keydown', function(e) {
            if (e.key === 'Enter' && e.ctrlKey) {
                startResearch();
            }
        });
    </script>
</body>
</html>
"""

# Flask routes
@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/research', methods=['POST'])
def research():
    try:
        data = request.get_json()
        query = data.get('query', '').strip()
        max_sources = data.get('max_sources', 5)
        
        if not query:
            return jsonify({'success': False, 'error': 'Query is required'})
        
        if research_agent is None:
            success, msg = initialize_agent()
            if not success:
                return jsonify({'success': False, 'error': msg})
        
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        try:
            result = loop.run_until_complete(research_agent.research(query, max_sources=max_sources))
        finally:
            loop.close()
        
        return jsonify({
            'success': True,
            'query': result.query,
            'report': result.report,
            'sources': [{
                'title': s.title,
                'url': s.url,
                'source_type': s.source_type.title(),
                'relevance_score': s.relevance_score
            } for s in result.sources],
            'citations': result.citations,
            'confidence_score': result.confidence_score,
            'research_time': result.research_time
        })
        
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})

@app.route('/status')
def status():
    try:
        if research_agent is None:
            success, msg = initialize_agent()
            if not success:
                return jsonify({'status': msg})
        
        status = research_agent.get_model_status()
        
        status_text = f""" MODEL STATUS
LLM Connected: {'✅ Yes' if status['llm_loaded'] else '❌ No'}
Search Available: {'✅ Yes' if status['search_available'] else '❌ No'}
"""
        
        if status['model_info']:
            info = status['model_info']
            status_text += f"""
Model: {info.get('model_name', 'Unknown')}
API Server: {info.get('api_url', 'Unknown')}
Server Type: {info.get('server_type', 'Unknown')}
"""
        
        return jsonify({'status': status_text})
    except Exception as e:
        return jsonify({'status': f'Error getting status: {str(e)}'})

@app.route('/history')
def history():
    try:
        if research_agent is None:
            return jsonify({'history': 'Agent not initialized'})
        
        history = research_agent.get_research_history()
        if not history:
            return jsonify({'history': ' No research history yet. Start by asking a question!'})
        
        history_text = f" Research History ({len(history)} queries)\n\n"
        for i, result in enumerate(history, 1):
            history_text += f"{i}. {result.query}\n"
            history_text += f"   Confidence: {result.confidence_score:.2f}\n"
            history_text += f"   Sources: {len(result.sources)}\n"
            history_text += f"   Time: {result.research_time:.1f}s\n\n"
        
        return jsonify({'history': history_text})
    except Exception as e:
        return jsonify({'history': f'Error getting history: {str(e)}'})

# Function to launch in Jupyter
def launch_jupyter_web_interface(port=7861):
    """Launch Flask web interface from Jupyter notebook"""
    
    print(" Initializing research agent for web interface...")
    init_success, init_message = initialize_agent()
    print(init_message)
    
    def run_flask():
        app.run(host='127.0.0.1', port=port, debug=False, use_reloader=False)
    
    # Start Flask in background thread
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    
    # Wait for server to start
    sleep(2)
    
    # Open browser automatically
    webbrowser.open(f'http://127.0.0.1:{port}')
    
    print(f"\n Web interface launched at: http://127.0.0.1:{port}")
    print("Interface is local only - not accessible from internet") 
    print("The server runs in the background - you can continue using Jupyter")
    print("To stop the server, restart the kernel")
    
    return flask_thread

# Launch the web interface
web_thread = launch_jupyter_web_interface(port=7861)

✅ Created directories
 Initializing research agent for web interface...
🤖 Initializing Local Research Agent with your Llama 3 model...
✅ Connected to your local Llama 3 model API!
   Model: Meta-Llama-3-8B-Instruct_Q4_K_M.gguf
   Server: http://127.0.0.1:8080
✅ Local LLM connected
✅ arXiv academic search available
✅ Wikipedia search available
✅ Search tools initialized
Research agent initialized successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7861
Press CTRL+C to quit



 Web interface launched at: http://127.0.0.1:7861
Interface is local only - not accessible from internet
The server runs in the background - you can continue using Jupyter
To stop the server, restart the kernel


127.0.0.1 - - [19/Sep/2025 16:33:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 16:33:21] "GET /favicon.ico HTTP/1.1" 404 -



🔍 Starting research on: artificial intelligence ethics
📚 Step 1: Searching for sources...
🔍 Searching for: artificial intelligence ethics
   ❌ Academic search failed: Page request resulted in HTTP 301: This study proposes an analysis of the different types of ethical approaches
involved in the ethics of AI, and situates their interests and limits. First,
the author introduces to the contemporary need for and meaning of ethics. He
distinguishes it from other registers of normativities and underlines its
inadequacy to formalization. He then presents a cartography of the landscape of
ethical theories covered by moral philosophy, taking care to distinguish
meta-ethics, normative ethics and applied ethics. In drawing up this overview,
the author questions the relationship between ethics and artificial
intelligence. The analysis focuses in particular on the main ethical currents
that have imposed themselves in the ways of doing digital ethics and AI in our
Western democracies. The author as

127.0.0.1 - - [19/Sep/2025 16:34:34] "POST /research HTTP/1.1" 200 -


✅ Research completed in 51.2 seconds
   Confidence Score: 0.47


127.0.0.1 - - [19/Sep/2025 16:35:02] "GET /history HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 16:35:05] "GET /status HTTP/1.1" 200 -
